In [4]:
#Day1
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough


model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

haiku_prompt = ChatPromptTemplate.from_template("Write a short Haiku about the programming language '{language}'.")      
haiku_chain = LLMChain(llm=model, prompt=haiku_prompt)

explain_prompt = ChatPromptTemplate.from_template(
    "Explain the following Haiku about a programming language:\n\n'{haiku}'"
)
explain_chain = LLMChain(llm=model, prompt=explain_prompt)

def chain_with_explanation(input_language):
    haiku_result = haiku_chain.invoke({"language": input_language})
    haiku = haiku_result['text'].strip()
    
    explanation_result = explain_chain.invoke({"haiku": haiku})
    explanation = explanation_result['text'].strip()
                                                                                                            
    return {
        "language": input_language,
        "haiku": haiku,
        "explanation": explanation
    }

if __name__ == "__main__":
    language_name = "Python"
    result = chain_with_explanation(language_name)
    print(f"Programming Language: {result['language']}\n")
    print(f"Haiku:\n{result['haiku']}\n")
    print(f"Explanation:\n{result['explanation']}")



Programming Language: Python

Haiku:
Python, sleek and fast
Code flows like a river stream
Efficient, powerful

Explanation:
This Haiku praises Python as a programming language that is sleek, fast, and efficient. It describes how coding in Python feels natural and smooth, like a river stream flowing effortlessly. The language is also portrayed as powerful and effective in getting tasks done efficiently.


In [3]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

# Initialize the LLM
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# 1. Create Examples for Few-Shot Learning
examples = [
    {
        "movie": "Inception",
        "response": (
            "Title: Inception\n"
            "Director: Christopher Nolan\n"
            "Main Cast: Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page, Tom Hardy\n"
            "Budget: $160 million\n"
            "Box Office Revenue: $836 million\n"
            "Genre: Science Fiction, Thriller\n"
            "Synopsis: A skilled thief is given a chance at redemption if he can successfully perform inception, the planting of an idea into someone's subconscious."
        ),
    },
    {
        "movie": "The Matrix",
        "response": (
            "Title: The Matrix\n"
            "Director: The Wachowskis\n"
            "Main Cast: Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, Hugo Weaving\n"
            "Budget: $63 million\n"
            "Box Office Revenue: $467 million\n"
            "Genre: Science Fiction, Action\n"
            "Synopsis: A computer hacker learns about the true nature of his reality and his role in the war against its controllers."
        ),
    },
    {
        "movie": "Titanic",
        "response": (
            "Title: Titanic\n"
            "Director: James Cameron\n"
            "Main Cast: Leonardo DiCaprio, Kate Winslet, Billy Zane, Kathy Bates\n"
            "Budget: $200 million\n"
            "Box Office Revenue: $2.2 billion\n"
            "Genre: Romance, Drama\n"
            "Synopsis: A young aristocrat falls in love with a kind but poor artist aboard the ill-fated R.M.S. Titanic."
        ),
    },
]

# 2. Define the Example Prompt Template
example_prompt = PromptTemplate(
    input_variables=["movie", "response"],
    template="Provide information about the movie '{movie}':\n{response}"
)

# 3. Create the Few-Shot Prompt Template
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="You are a movie expert. Always provide information about movies in the following format:\n"
           "Title: <Movie Title>\n"
           "Director: <Director Name>\n"
           "Main Cast: <List of Main Actors>\n"
           "Budget: <Budget Amount>\n"
           "Box Office Revenue: <Box Office Revenue>\n"
           "Genre: <Genre of the Movie>\n"
           "Synopsis: <Brief Synopsis>\n\nHere are some examples:",
    suffix="Provide information about the movie '{movie}':",
    input_variables=["movie"],
)

# 4. Build the Chain
movie_chain = LLMChain(llm=model, prompt=few_shot_prompt)

# 5. Example Usage
if __name__ == "__main__":
    movie_name = "The Godfather"  # Change this to any movie name
    result = movie_chain.run(movie=movie_name)
    print(result)


Title: The Godfather
Director: Francis Ford Coppola
Main Cast: Marlon Brando, Al Pacino, James Caan, Robert Duvall
Budget: $6 million
Box Office Revenue: $245 million
Genre: Crime, Drama
Synopsis: The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.


In [5]:
import os
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1, openai_api_key=openai_api_key)

examples = [
    {"movie_title": "Top Gun", "emojis": "🛩️👨‍✈️🔥"},
    {"movie_title": "The Godfather", "emojis": "👨‍👨‍👦🔫🍝"},
    {"movie_title": "Titanic", "emojis": "🚢💔🧊"},
    {"movie_title": "Jurassic Park", "emojis": "🦖🌴🚁"},
]

example_template = """
Movie Title: {movie_title}
Emojis: {emojis}
"""
example_prompt = PromptTemplate(input_variables=["movie_title", "emojis"], template=example_template)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Provide three emojis that represent the movie title given below.",
    suffix="Movie Title: {movie_title}\nEmojis:",
    input_variables=["movie_title"],
)

memory = ConversationBufferMemory(memory_key="chat_history", input_key="movie_title")

chain = LLMChain(
    llm=llm,
    prompt=few_shot_prompt,
    memory=memory,
)

response1 = chain.run("Inception")
print("Inception ->", response1)

response2 = chain.run("The Matrix")
print("The Matrix ->", response2)

recall_prompt = "What was the first movie I asked about?"
response3 = chain.run(recall_prompt)
print("Recall ->", response3)


Inception -> 🧠🌀🌌
The Matrix -> 🕶️💊💥
Recall -> 🛩️👨‍✈️🔥
